In [1]:
import sqlite3
import sqlalchemy

In [2]:
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
from sqlalchemy.types import Integer, Text
from sqlalchemy.schema import ForeignKey, Column
from sqlalchemy.engine import create_engine

In [3]:
base = declarative_base()
session = sessionmaker()
sess = session()
engine = create_engine('sqlite:///:memory:', echo=True)
sess.bind = engine

In [4]:
class Post(base):
    __tablename__ = 'POST'
    pk = Column('PK', Integer, primary_key=True)
    content = Column('CONTENT', Text, default='')
    tags = relationship('PH', back_populates='post', uselist=True)
    
class Hashtag(base):
    __tablename__ = 'HASHTAG'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, default='')
    count = Column('COUNT', Integer, default=0)
    posts = relationship('PH', back_populates='tag', uselist=True)
    
class PH(base):
    __tablename__ = 'PH'
    pk = Column('PK', Integer, primary_key=True)
    pfk = Column('PFK', Integer, ForeignKey('POST.PK'))
    hfk = Column('HFK', Integer, ForeignKey('HASHTAG.PK'))
    post = relationship('Post', back_populates='tags', uselist=False)
    tag = relationship('Hashtag', back_populates='posts', uselist=False)

In [5]:
base.metadata.create_all(engine)

2025-03-07 12:00:42,006 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:00:42,007 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POST")
2025-03-07 12:00:42,008 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 12:00:42,009 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POST")
2025-03-07 12:00:42,009 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 12:00:42,010 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("HASHTAG")
2025-03-07 12:00:42,010 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 12:00:42,010 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("HASHTAG")
2025-03-07 12:00:42,011 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 12:00:42,011 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PH")
2025-03-07 12:00:42,011 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 12:00:42,012 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PH")
2025-03-07 12:00:42,012 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 12:0

In [6]:
ph1 = PH()

ph1.post = Post(content='내용1')
ph1.tag = Hashtag(name='태그1')

In [8]:
sess.add(ph1)
sess.commit()

2025-03-07 12:01:21,446 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:01:21,456 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME", "COUNT") VALUES (?, ?)
2025-03-07 12:01:21,469 INFO sqlalchemy.engine.Engine [generated in 0.01313s] ('태그1', 0)
2025-03-07 12:01:21,474 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2025-03-07 12:01:21,478 INFO sqlalchemy.engine.Engine [generated in 0.00405s] ('내용1',)
2025-03-07 12:01:21,481 INFO sqlalchemy.engine.Engine INSERT INTO "PH" ("PFK", "HFK") VALUES (?, ?)
2025-03-07 12:01:21,481 INFO sqlalchemy.engine.Engine [generated in 0.00048s] (1, 1)
2025-03-07 12:01:21,482 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
ph1.post.content, ph1.tag.name
# INSERT Post 내용
# INSERT Hashtag 태그
# INSERT PH 내용,태그의 pk

2025-03-07 12:01:29,919 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:01:29,922 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE "PH"."PK" = ?
2025-03-07 12:01:29,923 INFO sqlalchemy.engine.Engine [generated in 0.00126s] (1,)
2025-03-07 12:01:29,926 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 12:01:29,926 INFO sqlalchemy.engine.Engine [generated in 0.00062s] (1,)
2025-03-07 12:01:29,928 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 12:01:29,928 INFO sqlalchemy.engine.Engine [generated in 0.00048s] (1,)


('내용1', '태그1')

In [10]:
p1 = sess.query(Post).one()

2025-03-07 12:01:38,062 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST"
2025-03-07 12:01:38,063 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()


In [11]:
p1.content, len(p1.tags), p1.tags[0].tag.name

2025-03-07 12:01:43,995 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 12:01:43,996 INFO sqlalchemy.engine.Engine [generated in 0.00088s] (1,)


('내용1', 1, '태그1')

In [12]:
p1.content, len(p1.tags), p1.tags[0].tag.name

('내용1', 1, '태그1')

In [13]:
p1.tags[0].tag.name = '새로운 태그1'

In [14]:
sess.dirty

IdentitySet([<__main__.Hashtag object at 0x107255f50>])

In [15]:
sess.commit()

2025-03-07 12:02:08,900 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "NAME"=? WHERE "HASHTAG"."PK" = ?
2025-03-07 12:02:08,901 INFO sqlalchemy.engine.Engine [generated in 0.00120s] ('새로운 태그1', 1)
2025-03-07 12:02:08,902 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
t2 = Hashtag(name='태그2')
sess.add(t2)
sess.commit()

2025-03-07 12:02:18,932 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:02:18,933 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME", "COUNT") VALUES (?, ?)
2025-03-07 12:02:18,933 INFO sqlalchemy.engine.Engine [cached since 57.48s ago] ('태그2', 0)
2025-03-07 12:02:18,935 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
p1.tags.append(PH(pfk=p1.pk, hfk=t2.pk))

2025-03-07 12:02:27,150 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:02:27,151 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 12:02:27,152 INFO sqlalchemy.engine.Engine [generated in 0.00039s] (1,)
2025-03-07 12:02:27,153 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 12:02:27,154 INFO sqlalchemy.engine.Engine [cached since 43.16s ago] (1,)
2025-03-07 12:02:27,155 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 12:02:27,155 INFO sqlalchemy.engine.Engine [generated in 0.00040s] (2,)


In [18]:
sess.dirty

IdentitySet([<__main__.Post object at 0x107243f50>])

In [19]:
sess.commit()

2025-03-07 12:02:40,713 INFO sqlalchemy.engine.Engine INSERT INTO "PH" ("PFK", "HFK") VALUES (?, ?)
2025-03-07 12:02:40,714 INFO sqlalchemy.engine.Engine [cached since 79.23s ago] (1, 2)
2025-03-07 12:02:40,714 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
len(p1.tags), p1.tags[-1].tag.name

2025-03-07 12:02:47,585 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:02:47,586 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 12:02:47,587 INFO sqlalchemy.engine.Engine [cached since 20.44s ago] (1,)
2025-03-07 12:02:47,588 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 12:02:47,589 INFO sqlalchemy.engine.Engine [cached since 63.59s ago] (1,)
2025-03-07 12:02:47,590 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 12:02:47,590 INFO sqlalchemy.engine.Engine [cached since 20.44s ago] (2,)


(2, '태그2')

In [21]:
p1.tags.pop(-1)

In [22]:
len(p1.tags)

1

In [23]:
sess.dirty

IdentitySet([<__main__.Post object at 0x107243f50>, <__main__.PH object at 0x10726f2d0>])

In [24]:
sess.commit()

2025-03-07 12:03:11,288 INFO sqlalchemy.engine.Engine UPDATE "PH" SET "PFK"=? WHERE "PH"."PK" = ?
2025-03-07 12:03:11,288 INFO sqlalchemy.engine.Engine [generated in 0.00059s] (None, 2)
2025-03-07 12:03:11,289 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
sess.query(PH).all()[1].pfk

2025-03-07 12:03:19,350 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:03:19,351 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH"
2025-03-07 12:03:19,351 INFO sqlalchemy.engine.Engine [generated in 0.00049s] ()


In [26]:
t3 = Hashtag(name='태그3')
sess.add(t3)
sess.commit()

2025-03-07 12:03:28,375 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME", "COUNT") VALUES (?, ?)
2025-03-07 12:03:28,376 INFO sqlalchemy.engine.Engine [cached since 126.9s ago] ('태그3', 0)
2025-03-07 12:03:28,377 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
p1.tags.append(PH(pfk=p1.pk, hfk=t3.pk))
sess.dirty

2025-03-07 12:03:34,919 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:03:34,920 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 12:03:34,920 INFO sqlalchemy.engine.Engine [cached since 67.77s ago] (1,)
2025-03-07 12:03:34,921 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 12:03:34,921 INFO sqlalchemy.engine.Engine [cached since 110.9s ago] (1,)
2025-03-07 12:03:34,922 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 12:03:34,922 INFO sqlalchemy.engine.Engine [cached since 67.77s ago] (3,)


IdentitySet([<__main__.Post object at 0x107243f50>])

In [28]:
sess.commit()

2025-03-07 12:03:41,353 INFO sqlalchemy.engine.Engine INSERT INTO "PH" ("PFK", "HFK") VALUES (?, ?)
2025-03-07 12:03:41,353 INFO sqlalchemy.engine.Engine [cached since 139.9s ago] (1, 3)
2025-03-07 12:03:41,354 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
base.registry.dispose()
sess.close()
sess.bind = engine

In [ ]:
class Post(base):
    __table__ = base.metadata.tables['POST']
    pk = base.metadata.tables['POST'].c['PK']
    content = base.metadata.tables['POST'].c['CONTENT']
    tags = relationship('PH', back_populates='post', uselist=True)
    def add(self, s, *tags):
        for tag in tags:
            t = s.query(Hashtag).filter(Hashtag.name==tag).all()
            if len(t) == 0:
                t = Hashtag(name=tag)
                s.add(t)
                s.commit()
            else:
                t = t[0]
            t.count += 1
            self.tags.append(PH(pfk=self.pk, hfk=t.pk))
        s.commit()
    def update(self, s, *tags):
        print('어떻게?')         
        s.commit()

class Hashtag(base):
    __table__ = base.metadata.tables['HASHTAG']
    pk = base.metadata.tables['HASHTAG'].c['PK']
    name = base.metadata.tables['HASHTAG'].c['NAME']
    count = base.metadata.tables['HASHTAG'].c['COUNT']
    posts = relationship('PH', back_populates='tag', uselist=True)
    def plus(self):
        self.count += 1
    def minus(self):
        self.count -= 1

class PH(base):
    __table__ = base.metadata.tables['PH']
    pk = base.metadata.tables['PH'].c['PK']
    pfk = base.metadata.tables['PH'].c['PFK']
    hfk = base.metadata.tables['PH'].c['HFK']
    post = relationship('Post', back_populates='tags', uselist=False)
    tag = relationship('Hashtag', back_populates='posts', uselist=False)

In [31]:
new_p = Post(content='새로운 게시글')

In [32]:
sess.add(new_p)
sess.commit()

2025-03-07 12:19:41,311 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:19:41,313 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2025-03-07 12:19:41,314 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ('새로운 게시글',)
2025-03-07 12:19:41,315 INFO sqlalchemy.engine.Engine COMMIT


In [33]:
new_p.add(sess, '새로운 태그1', '새로운 태그2', '새로운 태그3')

2025-03-07 12:19:42,203 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:19:42,205 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?
2025-03-07 12:19:42,206 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ('새로운 태그1',)
2025-03-07 12:19:42,208 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "COUNT"=? WHERE "HASHTAG"."PK" = ?
2025-03-07 12:19:42,209 INFO sqlalchemy.engine.Engine [generated in 0.00049s] (1, 1)
2025-03-07 12:19:42,210 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 12:19:42,210 INFO sqlalchemy.engine.Engine [generated in 0.00029s] (2,)
2025-03-07 12:19:42,211 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 12:19:42,212 INFO 

In [34]:
new_p.content, len(new_p.tags), new_p.tags[0].tag.name, new_p.tags[0].tag.count

2025-03-07 12:19:43,227 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 12:19:43,229 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 12:19:43,229 INFO sqlalchemy.engine.Engine [cached since 1.019s ago] (2,)
2025-03-07 12:19:43,231 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 12:19:43,231 INFO sqlalchemy.engine.Engine [cached since 1.02s ago] (2,)
2025-03-07 12:19:43,232 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 12:19:43,232 INFO sqlalchemy.engine.Engine [generated in 0.00047s] (1,)


('새로운 게시글', 3, '새로운 태그1', 1)

In [35]:
new_p.content, len(new_p.tags), new_p.tags[1].tag.name, new_p.tags[1].tag.count

2025-03-07 12:19:44,741 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 12:19:44,742 INFO sqlalchemy.engine.Engine [cached since 1.51s ago] (4,)


('새로운 게시글', 3, '새로운 태그2', 1)